<font color='#DF1525'>  

## Disclaimer:  
  
A ideia de criar um notebook organizacional foi herdada do P3 feito por Giovanni Rozatti (em conjunto com Daniel G. Terra e Rafael B. Zanfolin) no semestre anterior, o repositório para esse notebook daquele projeto se encontra no link a seguir:`https://github.com/VonIgnia/C.Dados-P3/blob/master/Classificador2/_Fun%C3%A7%C3%B5es.ipynb`
- A ideia original pertence a Daniel Terra, e sua execução(no Projeto 3) foi feita em conjunto com Giovanni Rozatti
- A maior parte das funções nesse notebook é inédita, apesar de bastante simlares àquele, isso ocorre pois as mesmas fontes (vistas em [Links e Referências](Links%20e%20Refer%C3%AAncias.md)) foram consultadas
- Partes diretmente extraídas daquele serão antecedidas por outro disclaimer  
</font>

### Objetivos do Notebook:
- O principal propósito desse notebook é a hospedagem e organização das funções que serão utilizados nos outros notebooks desse projeto.  
- Outro propósito é uma breve explicação do funcionamento individual de cada uma delas

___
### Imports iniciais
- Importa bibliotecas e funções com que trabalharemos ao longo desse notebook

In [1]:
#biblioteecas para manipulação de dados
import pandas as pd
import numpy as np

In [2]:
#bibliotecas para deixar determinados trechos "user friendly" e interativos
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
# Biblioteca para criação de gráficos e visualizações de dados
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn

In [4]:
# Biblioteca de Machine Learning Open source para python
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import *

___
### Dados
- Importa nossa base de dados

In [5]:
Dados=pd.read_csv('cars_Brazil_data.csv')

- Optamos por trabalhar apenas com carros que apareçam no Dataset no mínimo 500 vezes, esse valor foi escolhido de forma a separar o Dataset individual de cada carro em teste e treinamento, com 10% (no mínimo 50 linhas) do Dataset sendo direcionado para teste.   
    - <font color="#F02510"> **Observação 04/06: Talvez seja viável reduzir esse limite para 300, em contraponto o limite de 500 já fornece uma base de dados extensa o bastante e informações que parecem ser suficientes para elaborar uma conclusão.** </font>
    - Colunas de caracerísticas qualitativas nominais devem ser transformadas em variáveis Dummy e/ou Hot End, caso contrário a predição não pode ser feita

In [6]:
#separa carros com mais de 500 ocorrências no Dataframe
Mais_de_500 = Dados['nome'].value_counts() > 500
Mais_de_500 = list(Mais_de_500.loc[Mais_de_500].index)
Dados = Dados.loc[Dados['nome'].isin(Mais_de_500),:];

#Converte as variáveis quali em quanti através do uso de Dummy Variables:
Dummies = pd.get_dummies(Dados.drop(columns=['nome']), prefix=['marca', 'categoria','loc']).dropna()

#### Separando 10% do Dataframe global para treinamento

In [7]:
# separa características dos carros, que podem ser relevantes no contexto de uma regrssão e \n
# elimina as características irrelevantes assim como a coluna com resultado esperado;
inputs = Dummies.drop(columns=["valor"])

#separa o resultado esperado, devemos calcular a previsão mais próxima a eles;
target = Dummies["valor"]

#separando dataframes teste e treino
X_train,X_test,Y_train,Y_test=tts(inputs,target,test_size=0.9)

#### Separando 10% do Dataframe de cada carro para treinamento:

In [8]:
#Criando dicionários, pois eles são melhores para chamar um grande número de valores armazenados
Carro         = {}
Carro_inputs  = {}
Carro_target  = {}
Carro_X_train = {}
Carro_X_test  = {}
Carro_Y_train = {}
Carro_Y_test  = {}
Carro_Y_predict = {}

for x in Mais_de_500:
    Carro[x] = Dummies.loc[Dados['nome'] == x]
        
    Carro_inputs[x] = Carro[x].drop(columns=["valor"])
    Carro_target[x] = Carro[x]["valor"]
        
    #separando dataframes teste e treino
    Carro_X_train[x],Carro_X_test[x],Carro_Y_train[x],Carro_Y_test[x]=tts(Carro_inputs[x],Carro_target[x],test_size=0.9)

In [9]:
#Usar essa informação com a função interact do ipywidgets
def Separa_dataframe_por_carro(x):
    return (Carro_X_train[x],Carro_X_test[x],Carro_Y_train[x],Carro_Y_test[x])

___
### Pré Processamento de Modelos de Regressão e Classificação do SKLearn:

#### Linear Regression (da Biblioteca SKLearn)

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
def Linear_Regression_individual(x):
    ### Declarando o modelo como variável global, para que ele possa ser chamado novamente sem perda ou alteração de valor
    global model_1
    
    ### Escolhendo o modelo na biblioteca SciKitLearn
    model_1 = LinearRegression()
    
    ### Fazendo o ajuste do modelo
    model_1.fit(Separa_dataframe_por_carro(x)[0],Separa_dataframe_por_carro(x)[2])
    
    ### Usando o modelo ajustado para elaborar uma predição
    Carro_Y_predict = {}
    Carro_Y_predict[x] = model_1.predict(Carro_X_test[x])
    
    ### Calculando as métricas que determinam a qualidade do modelo
    r_2 = model_1.score(Separa_dataframe_por_carro(x)[1],Separa_dataframe_por_carro(x)[3])
    MSE  = mean_squared_error (Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    MAE  = mean_absolute_error(Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    
    #Função de plotagem com seaborn extraída de:`https://onestopdataanalysis.com/scatter-plot-python/`
    sn.set(color_codes=True)
    plt.figure(1, figsize=(6, 4))
    plt.title("Comparação Valor real X Previsão")
    scatter = sn.scatterplot(Separa_dataframe_por_carro(x)[3],Carro_Y_predict[x], color="#D02515")
    scatter.set(xlabel="Valor Real", ylabel="Previsão")
    
    ### Imprimindo as métricas
    print ("R-squared: {0:.4f}".format(r_2))
    print ("Mean Squared Error: {0:.2f}" .format(MSE))
    print ("Mean Absolute Error: {0:.2f}".format(MAE))
    
    return

___
#### Regression Tree (da Biblioteca SKLearn)

In [12]:
from sklearn.tree import DecisionTreeRegressor

- Avisos:
    - Apesar do valor default do `random_state` ser `None`, ainda haviam varaiações de resultados quando o modelo era rodado(sem alterar a base de treinamento), isso foi solucionado ao trocar o valor  do `random_state` por 0.

In [13]:
def Regression_Tree_individual(x):
    ### Declarando o modelo como variável global, para que ele possa ser chamado novamente sem perda ou alteração de valor
    global model_2
    
    ### Escolhendo o modelo na biblioteca SciKitLearn
    model_2 = DecisionTreeRegressor(max_depth=12,random_state=0)
    
    ### Fazendo o ajuste do modelo
    model_2.fit(Separa_dataframe_por_carro(x)[0],Separa_dataframe_por_carro(x)[2])
    
    ### Usando o modelo ajustado para elaborar uma predição
    Carro_Y_predict = {}
    Carro_Y_predict[x] = model_2.predict(Carro_X_test[x])
    
    ### Calculando as métricas que determinam a qualidade do modelo
    r_2 = model_2.score(Separa_dataframe_por_carro(x)[1],Separa_dataframe_por_carro(x)[3])
    MSE  = mean_squared_error (Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    MAE  = mean_absolute_error(Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    
    ### Função de plotagem com seaborn extraída de:`https://onestopdataanalysis.com/scatter-plot-python/`
    sn.set(color_codes=True)
    plt.figure(1, figsize=(6, 4))
    plt.title("Comparação Valor real X Previsão")
    scatter = sn.scatterplot(Separa_dataframe_por_carro(x)[3],Carro_Y_predict[x], color="#D02515")
    scatter.set(xlabel="Valor Real", ylabel="Previsão")
    
    ### Imprimindo as métricas
    print ("R-squared: {0:.4f}".format(r_2))
    print ("Mean Squared Error: {0:.2f}" .format(MSE))
    print ("Mean Absolute Error: {0:.2f}".format(MAE))
    
    return

___
#### Random Forest Regressor (da Biblioteca SKLearn)

In [14]:
from sklearn.ensemble import RandomForestRegressor

- Avisos:
    - Apesar do valor default do `random_state` ser `None`, ainda haviam varaiações de resultados quando o modelo era rodado(sem alterar a base de treinamento), isso foi solucionado ao trocar o valor  do `random_state` por 0.

In [15]:
def Random_Forest_individual(x):
    ### Declarando o modelo como variável global, para que ele possa ser chamado novamente sem perda ou alteração de valor
    global model_3
    
    ### Escolhendo o modelo na biblioteca SciKitLearn
    model_3 = RandomForestRegressor(max_depth=12,n_estimators=50,random_state=0)
    
    ### Fazendo o ajuste do modelo
    model_3.fit(Separa_dataframe_por_carro(x)[0],Separa_dataframe_por_carro(x)[2])

    ### Usando o modelo ajustado para elaborar uma predição
    Carro_Y_predict = {}
    Carro_Y_predict[x] = model_3.predict(Carro_X_test[x])
    
    ### Calculando as métricas que determinam a qualidade do modelo
    r_2 = model_3.score(Separa_dataframe_por_carro(x)[1],Separa_dataframe_por_carro(x)[3])
    MSE  = mean_squared_error (Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    MAE  = mean_absolute_error(Separa_dataframe_por_carro(x)[3], Carro_Y_predict[x])
    
    ### Função de plotagem com seaborn extraída de:`https://onestopdataanalysis.com/scatter-plot-python/`
    sn.set(color_codes=True)
    plt.figure(1, figsize=(6, 4))
    plt.title("Comparação Valor real X Previsão")
    scatter = sn.scatterplot(Separa_dataframe_por_carro(x)[3],Carro_Y_predict[x], color="#D02515")
    scatter.set(xlabel="Valor Real", ylabel="Previsão")
    
    ### Imprimindo as métricas
    print ("R-squared: {0:.4f}".format(r_2))
    print ("Mean Squared Error: {0:.2f}" .format(MSE))
    print ("Mean Absolute Error: {0:.2f}".format(MAE))
    
    return

___
<font color="#DF1525">  
## Disclaimer:

Os trechos de código da seção a seguir foram extraídos de e/ou são fortemente inspirados em:
- `https://github.com/VonIgnia/C.Dados-P3/blob/master/Classificador2/_Classificadores.ipynb` 
</font>  
#### Comparação de Modelos:

In [16]:
def Modelos_de_Regressão(tipo, k=False, arvore=False, floresta=False):
    # k representa a profundidade da DecisionTree e/ou Random Forest
    
    ### Escolha do modelo a ser usado na predição
    if arvore:
        model = tipo(max_depth=k,random_state=0)
    elif floresta:
        model = tipo(max_depth=k,n_estimators=50,random_state=0)
    else:
        model = tipo()
    
    ### Fazendo o ajuste do modelo, independente de qual modelo seja
    model.fit(X_train,Y_train)
    
    ### Usando o modelo ajustado para elaborar uma predição
    Y_predict = model.predict(X_test)
    
    ### Calculando as métricas que determinam a qualidade do modelo
    r_2  = model.score(X_test,Y_test)
    MSE  = mean_squared_error (Y_test,Y_predict)
    MAE  = mean_absolute_error(Y_test,Y_predict)

    ### Função de plotagem com seaborn extraída de:`https://onestopdataanalysis.com/scatter-plot-python/`
    sn.set(color_codes=True)
    plt.figure(1, figsize=(6, 4))
    plt.title("Comparação Valor real X Previsão")
    scatter = sn.scatterplot(Y_test,Y_predict, color="#D02515")
    scatter.set(xlabel="Valor Real", ylabel="Previsão")
    
    ### Imprimindo as métricas que determinam a qualidade do modelo
    print ("R-squared: {0:.4f}".format(r_2))
    print ("Mean Squared Error: {0:.2f}" .format(MSE))
    print ("Mean Absolute Error: {0:.2f}".format(MAE))
    
    return

In [17]:
#Qualquer coisa para causar erro na rodagem
#Como as features a seguir não foram totalmente implementadas, essa célula impede a sua rodagem

#### Criando um carro, fora do Dataframe e estimando o seu valor (produto para o usuário final)
Apesar do conjunto de dados ser bastante extenso, ele não é completo, caso o usuário final deseje consultar o preço de um carro baseado em suas características ele não procuraria por esse carro na base de dados e pelo resultado obtido em um target, e é para isso que a seção a seguir foi criada

In [18]:
#from IPython.display import display

In [19]:
#def Cria_carro_novo(nome, marca, categoria, loc, ano, km_rodados,modelo):
#    info = {'nome':  [nome],
#            'marca': [marca],
#            'categoria': [categoria],
#            'loc': [loc],
#            'ano': [ano],
#            'km_rodados':[km_rodados]}
#    Dataframe = pd.DataFrame (info, columns = ['nome','marca', 'categoria', 'loc', 'ano', 'km_rodados'])
#    Dataframe.join(Dummies.drop(columns=['ano','km_rodados']))
#    Dataframe = pd.get_dummies(Dataframe.drop(columns=['nome']), prefix=['marca', 'categoria','loc'],dummy_na=False)
#    Dataframe.join(pd.get_dummies(Dados['marca']))
#    print(Dataframe)
#
#    result = modelo.predict(Dataframe)# ajustar o número de colunas
#    print(result)
#    
#    return

In [20]:
#fabricante = list(Dados['marca'].astype('category').values.categories)
#tipo = list(Dados['categoria'].astype('category').values.categories)
#localização = list(Dados['loc'].astype('category').values.categories)
#modelos = {"Regressão Linear": model_3,
#           "Árvore de regressões": model_3,
#           "Random Forest":model_3} 

In [21]:
#Calculadora = interactive(Cria_carro_novo, {'manual': True}, nome=Mais_de_500,
#                          marca=fabricante, categoria=tipo,loc=localização,
#                          ano=(1982,2020,1), km_rodados=(0,500000,10000),modelo=modelos)

In [22]:
#display(Calculadora)
# o erro na previsão se deve a diferença entre o número de colunas
# sendo que elas são 50 no treinamento ou fit do modelo e 5 no dataframe criado